# Custom function-calling chain

`langchain` comes with built-in `create_openai_tools_chain` and `create_openai_functions_chain` constructors for building tool chains that make use of function-calling APIs (see [Quickstart](/docs/use_cases/tool_use/quickstart) for an example of how to use these). To better understand what's happening under the hood of these, let's built a custom function-calling chain from scratch.

## Setup

We'll need to install the following packages:

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

And set these environment variables:

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# If you'd like to use LangSmith, uncomment the below
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Create a tool

First, we need to create a tool to call. For this example, we will create a custom tool from a function. For more information on all details related to creating custom tools, please see [this guide](/docs/modules/agents/tools/).

In [1]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [3]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

## Create and bind OpenAI functions

For simplicity we'll use OpenAI's function calling API instead of the newer tools calling/parallel function calling API.

First we need to define our OpenAI functions. `langchain` comes with utilities for converting any `langchain` Tool into an OpenAI function:

In [25]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(multiply)]

In [26]:
functions

[{'name': 'multiply',
  'description': 'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.',
  'parameters': {'title': 'multiplySchemaSchema',
   'type': 'object',
   'properties': {'first_int': {'title': 'First Int', 'type': 'integer'},
    'second_int': {'title': 'Second Int', 'type': 'integer'}},
   'required': ['first_int', 'second_int']}}]

Now we'll bind our functions to our model, meaning the functions will be passed in as part of the payload to the model each time it is invoked. In this case we'll also bind `function_call`, which will force the OpenAI model to always return inputs for the `multiply` function.

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
model_with_functions = model.bind(
    functions=functions, function_call={"name": "multiply"}
)

## Chaining with output parser

We'll chain our model together with a `JsonOutputFunctionsParser`, which returns just the `functions` part of the model output (which is in JSON) as a dictionary. We'll specify `args_only=True` in this case so that only the function arguments and not the function name is returned, since we know which function the arguments are for:

In [28]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = model_with_functions | JsonOutputFunctionsParser(args_only=True)
chain.invoke("what is thirteen times 4")

{'first_int': 13, 'second_int': 4}

If we specified `args_only=False`, our chain output would look like:

In [30]:
(model_with_functions | JsonOutputFunctionsParser(args_only=False)).invoke(
    "what is thirteen times 4"
)

{'arguments': {'first_int': 13, 'second_int': 4}, 'name': 'multiply'}

## Adding prompt

Suppose we wanted to add some additional instructions to the model on each call. We can do this by including a prompt at the beginning of our chain:

In [31]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Always call function `multiply` with the smaller number passed in first.",
        ),
        ("user", "{input}"),
    ]
)

chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [32]:
chain.invoke({"input": "what is thirteen times 4"})

{'first_int': 4, 'second_int': 13}

## Invoking tool

And if we want our chain to actually call the tool once the model had determined the tool inputs, we can easily add that to our chain as well:

In [33]:
chain_with_tool = chain | multiply

In [34]:
chain_with_tool.invoke({"input": "what is thirteen times 4"})

52

### Multiple tools

Suppose we have multiple tools we want the chain to be able to choose from:

In [81]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

With function calling, we can do this like so:

In [82]:
tools = [multiply, add, exponentiate]
model_with_functions = model.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

chain = model_with_functions | JsonOutputFunctionsParser(args_only=False)
chain.invoke("what 17 cubed")

{'arguments': {'base': 17, 'exponent': 3}, 'name': 'exponentiate'}

If we want to run the model selected tool, we can do so using a function that returns the tool based on the model output. Specifically, our function will action return it's own subchain that gets the "arguments" part of the model output and passes it to the chosen tool:

In [83]:
from operator import itemgetter


def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool


chain = model_with_functions | JsonOutputFunctionsParser(args_only=False) | tool_chain
chain.invoke("what's 17 cubed")

4913